# ORIE 4330/5330 Project Part 2

This notebook is meant to get you started on part 2 of your term project. It will read all of the input files and store them as dataframes. In every cell, there should be comments that explain what the file being read containts. In addition, we import the class `PrelimExamAssignment` from the file `assign_rooms.py`. The class contains functions that you will have to complete in order to implement and solve the given constraints in `gurobipy`. You do not have to use the file/class if you prefer not to, just make sure you produce a feasible assignment of exams to rooms, by coding and solving the given model. The final output should be a datastructure(list, dataframe, whatever you prefer), that has the indices of $x$ variables that have value 1. 

Note that you are given an assignment of exams to days/slots similar to the one you had to produce for part 1. If you would rather use the one you created, you can, although you might need to edit some of the given code to parse it.

In [1]:
# automatically reloads the module if source file is changed
%load_ext autoreload
%autoreload 2

In [2]:
# imports packages we will use 
import numpy as np
import pandas as pd
import importlib
from datetime import datetime, timedelta
from gurobipy import *
from assign_rooms import PrelimExamAssignment

## Load Dataframes

In [3]:
# path for input data
input_data_path = '../Data/'

In [4]:
# creates the prelim exam df with prelims to schedule
# get the prelim schedule to the input format

# df with avail exam dates
# the set of values  will be your set of days D on which you can schedule exams. 
exam_dates = (pd.read_csv(f'{input_data_path}avail_prel_dates.csv')
             .reset_index().set_index('exam_dates').to_dict()['index'])
# number of slots per day
K = 2

# df with prelim exams requested
exams = (pd.read_csv(f'{input_data_path}prelim_exams.csv'))

# load prelim schedule from part 1
exams_schedule = (pd.read_csv(f'{input_data_path}prelim_date_schedule.csv'))

exams_schedule['d'] = exams_schedule['date'].apply(lambda x: exam_dates[x])

exams = (exams.merge(exams_schedule, on =['exam_id'])
    .drop(columns = ['course_x','date','prefdate','prefdate2','prefdate3'])
    .rename(columns={'course_y':'course','slot':'k'}))
exams

,exam_id,acadorg,enrollment,modality,course,k,d
0,1-AEM-2210-LEC-2634167-1,AEM,260,Online,AEM 2210,0,4
1,1-AEM-2210-LEC-2634167-2,AEM,260,Online,AEM 2210,1,12
2,1-AEM-2225-LEC-2634167-1,AEM,50,Online,AEM 2225,0,5
3,1-AEM-2225-LEC-2634167-2,AEM,50,Online,AEM 2225,0,13
4,1-AEM-2240-LEC-3778494-1,AEM,270,In person,AEM 2240,0,3
...,...,...,...,...,...,...,...
225,1-STSCI-1380-LEC-1757307-1,STSCI,64,In person,STSCI 1380,0,3
226,1-STSCI-1380-LEC-1757307-2,STSCI,64,In person,STSCI 1380,1,13
227,1-STSCI-2150-LEC-1319792-1,STSCI,140,In person,STSCI 2150,0,4
228,1-STSCI-2150-LEC-1319792-2,STSCI,140,Online,STSCI 2150,1,10


In [5]:
# Create the rooms dataframe
rooms = (pd.read_csv(f'{input_data_path}rooms.csv'))
rooms

,room_id,capacity,building,room
0,Morrison Hall-342,9,Morrison Hall,342
1,Physical Sciences Building-401,9,Physical Sciences Building,401
2,Rockefeller Hall-102,8,Rockefeller Hall,102
3,Olin Hall-128,9,Olin Hall,128
4,Baker Laboratory-G02,8,Baker Laboratory,G02
...,...,...,...,...
85,Sibley Hall-235,65,Sibley Hall,235
86,Statler Hall Auditorium-185,76,Statler Hall Auditorium,185
87,Schwartz Ctr Performing Arts-111,78,Schwartz Ctr Performing Arts,111
88,Bailey Hall-101,130,Bailey Hall,101


In [6]:
# Read the acadorg buidling distance matrix
acadorg_dist = pd.read_csv(f'{input_data_path}acadorg_dist.csv', index_col=0)
acadorg_dist.head()

,Morrison Hall,Physical Sciences Building,Rockefeller Hall,Olin Hall,Baker Laboratory,White Hall,Weill Hall,Riley-Robb Hall,Plant Science Building,Warren Hall,...,Kennedy Hall,Milstein Hall,Phillips Hall,Biotechnology,Klarman Hall,Uris Library,Anabel Taylor Hall,Sibley Hall,Schwartz Ctr Performing Arts,Bailey Hall
Acadorgs,,,,,,,,,,,,,,,,,,,,,
ILR,0.600392,0.191163,0.131026,0.213576,0.211481,0.310491,0.179837,0.509471,0.217869,0.242397,...,0.106786,0.306846,0.193601,0.143776,0.166404,0.228371,0.283603,0.300877,0.415663,0.145762
MSE,0.768437,0.429926,0.370194,0.105741,0.449989,0.448401,0.391308,0.666405,0.469326,0.508340,...,0.376746,0.505335,0.110195,0.334061,0.363274,0.279056,0.107020,0.484199,0.142019,0.417143
AEM,0.445776,0.244710,0.240273,0.455960,0.249645,0.430382,0.160390,0.382075,0.062460,0.000000,...,0.135632,0.359423,0.410154,0.204719,0.301198,0.434538,0.525991,0.376107,0.650349,0.153111
CHEM,0.692658,0.020750,0.083001,0.357541,0.000059,0.183860,0.316642,0.620729,0.278001,0.249645,...,0.193334,0.114992,0.394386,0.314952,0.097671,0.247618,0.414730,0.126976,0.578668,0.110959
ORIE,0.649347,0.450051,0.387875,0.206357,0.470781,0.519088,0.313416,0.546440,0.406572,0.457280,...,0.342301,0.551066,0.082016,0.257882,0.402623,0.366472,0.238159,0.536747,0.239878,0.406621


In [7]:
# Read the building buidling distance matrix
dist = pd.read_csv(f'{input_data_path}buildings_dist.csv', index_col=0)
dist.head()

,Morrison Hall,Physical Sciences Building,Rockefeller Hall,Olin Hall,Baker Laboratory,White Hall,Weill Hall,Riley-Robb Hall,Plant Science Building,Warren Hall,...,Kennedy Hall,Milstein Hall,Phillips Hall,Biotechnology,Klarman Hall,Uris Library,Anabel Taylor Hall,Sibley Hall,Schwartz Ctr Performing Arts,Bailey Hall
Morrison Hall,0.000000,0.684845,0.664750,0.773209,0.692658,0.867034,0.420573,0.102957,0.416633,0.445776,...,0.525352,0.804689,0.660057,0.464509,0.725549,0.828247,0.837567,0.819629,0.889184,0.586574
Physical Sciences Building,0.684845,0.000000,0.062251,0.338360,0.020750,0.185802,0.302028,0.610824,0.268815,0.244710,...,0.178271,0.128547,0.373643,0.297855,0.082539,0.234214,0.396508,0.136246,0.559373,0.099358
Rockefeller Hall,0.664750,0.062251,0.000000,0.282155,0.083001,0.204554,0.263177,0.584533,0.249718,0.240273,...,0.141969,0.177748,0.311421,0.249907,0.061634,0.201613,0.343565,0.176209,0.502183,0.087859
Olin Hall,0.773209,0.338360,0.282155,0.000000,0.357541,0.342742,0.367137,0.674295,0.427648,0.455960,...,0.320362,0.403243,0.137407,0.313952,0.265707,0.173671,0.070338,0.380729,0.221176,0.342415
Baker Laboratory,0.692658,0.020750,0.083001,0.357541,0.000000,0.183860,0.316642,0.620729,0.278001,0.249645,...,0.193334,0.114992,0.394386,0.314952,0.097671,0.247618,0.414730,0.126976,0.578668,0.110959


## Solve

Code and solve the model. Since the model will be run on every day/slot combination, you will want to filter the exams and run the model on the exams that have day/slot values equalto the combination of the current iteration. The code bellow loops over all the day/slot combinations found in the exam dateframe, and creates a `PrelimExamAssignment` object in order to assign the rooms. It will also create a dictionary entry that has key current day/slot and value the return value of the `solve()` method of `PrelimExamAssignment`, so assuming that the method return the indices of the $x$ variables that have value $1$ in the model, it will be sufficient as your final output. <br><br>
 
Feel free to change any part of the code, or not use it at all. 

In [8]:
# code and solve the model 
room_assignment = {}
for (d,k) in list(exams.groupby(['d','k']).count().index):
    print(d,k)
    slot_exams = exams[(exams.d == d) & (exams.k == k)]
    assign = PrelimExamAssignment(slot_exams, exam_dates, rooms, acadorg_dist, dist, 10)
    assign.build_model()        
    room_assignment[(d,k)] = assign.solve()

1 0
Using license file /home/sntan/gurobi.lic
Academic license - for non-commercial use only - expires 2022-09-01
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00
1 1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics

  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  0.000000000e+00
7 1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  0.000000000e+00
8 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linu

Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  0.000000000e+00
14 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual

In [9]:
room_assignment

{(1, 0): None,
 (1, 1): None,
 (2, 0): None,
 (2, 1): None,
 (3, 0): None,
 (3, 1): None,
 (4, 0): None,
 (4, 1): None,
 (5, 0): None,
 (5, 1): None,
 (6, 0): None,
 (6, 1): None,
 (7, 0): None,
 (7, 1): None,
 (8, 0): None,
 (8, 1): None,
 (9, 0): None,
 (9, 1): None,
 (10, 0): None,
 (10, 1): None,
 (11, 0): None,
 (11, 1): None,
 (12, 0): None,
 (12, 1): None,
 (13, 0): None,
 (13, 1): None,
 (14, 0): None,
 (14, 1): None,
 (15, 0): None,
 (15, 1): None,
 (16, 0): None,
 (16, 1): None,
 (17, 0): None,
 (17, 1): None,
 (18, 1): None}